# Projekt założenia
symulator zdarzeniowy M/M/1/N
    - średni czas oczekiwania na obsługę
    - prawdopobieństwo blokady
porównać zmodelem analitycznym:
    - dla N = 51
    - różnych wartości obciążenia: 0.01, 0.05, 0.1

średni czas oczekiwania - > histogram


In [206]:
# https://pyshark.com/poisson-distribution-and-poisson-process-in-python/#poisson-distribution-example-in-python
import scipy
import numpy as np
import matplotlib
from enum import Enum
import uuid

czas w serwerze to jest "usługa" 

In [207]:
# global_time = 0
n = 51 # max order in a system
# zmienna lososowa opisująca czas między zgłoszeniami
la = 3.35 # 1.0478 # 0.977 #  # intensywność napływu
# time_between_packet = 1 / la
# zmienna losowa opisuje czas pomiędzy zgłoszeniami k przez serwer
mu = 3
# time_of_service = 1 / mu # intensywność obsługi
time_of_simulation = 10
np.random.seed(12)

# https://github.com/amasl2048/queue-mm1/blob/master/qtest.cpp
roh = la / mu
rn = pow(roh, n + 1)
p0 = (1 - roh) / (1 - rn)         # free queue probability
pb = p0 * rn / roh                

navg = roh / (1-roh) - (n+1) * rn/(1-rn)  
tavg = navg / la                    
u    = mu * (1 - p0)                

out = '''
pb   = %.4f    # block probability
navg = %.4f    # average orders in a system
tavg = %.4f    # average time in a system
u    = %.4f    # throughput
'''
print(out % (pb, navg, tavg, u))


pb   = 0.1048    # block probability
navg = 42.5966    # average orders in a system
tavg = 12.7154    # average time in a system
u    = 2.9989    # throughput



In [208]:
class EventType(Enum):
    IN_EVENT = "IN_EVENT"
    OUT_EVENT = "OUT_EVENT"

# slajd 91, 

class Event():
    def __init__(self, time: float, event_type, client_id = None):
        self.time = time
        self.event_type = event_type
        if client_id == None:
            self.client_id = str(uuid.uuid4())
        else:
            self.client_id = client_id
    
    def __str__(self):
        mes = f"{self.client_id}, {self.event_type}, {self.time}"
        return mes
    
    def set_time_in_queue(self, time_in_queue):
        self.time_in_queue = time_in_queue
    
    def get_event_stats(self):
        return {
            'oc_time': self.time,
            # 'qu_time': self.time_in_queue,
            'client_id': self.client_id,
            'event_type': self.event_type
        }

In [209]:
class Server:
    def __init__(self):
        self.all_clients =0 
        self.all_clients_in_queue = 0
        self.client_in_server = False
        self.clients_number_in_queue = 0 
        self.client_in_server = None
        self.clients_in_queue = []
        self.buffor = n
    
    def get_server_stats(self):
        return self.client_in_server, self.clients_number_in_queue

    def is_server_empty(self):
        return not self.clients_in_server

    def is_queue_empty(self):
        return not self.clients_in_queue

    def is_place_for_new_event(self):
        return self.clients_number_in_queue < self.buffor

    def add_client_to_server(self, client_id, timestamp):
        self.all_clients += 1
        self.client_in_server = True
        self.clients_in_server = (client_id, timestamp)

    def add_client_to_queue(self, client_id, timestamp):
        self.all_clients_in_queue += 1
        self.clients_number_in_queue += 1
        self.clients_in_queue.insert(0, (client_id, timestamp))

    def get_queue_first_client(self):
        return self.clients_in_queue[-1]

    def delete_client_from_queue(self, client_id):
        self.clients_number_in_queue -= 1
        self.clients_in_queue = list(
            filter(
                lambda client: client[0] != client_id,
                self.clients_in_queue))

    def delete_client_from_server(self):
        self.client_in_server = False
        self.client_in_server = None




In [229]:
import time

class Simulation:
    def __init__(self):
        # marge
        self.events = []
        self.clients_stats = []
        self.server = Server()
        self.clients = 0 
        self.current_time = 0 


    def handle_event(self):
        current_event = self.events.pop()
        self.current_time = current_event.time
        # check buffor
        if current_event.event_type == EventType.IN_EVENT:
            self.events.append(Event(self.current_time + self.generate_service(),  EventType.OUT_EVENT, current_event.client_id))
        self.events =  self.sort_events()
        # print(self.current_time)
            

    def init_event(self):
        self.events.append(Event(self.current_time + self.generate_interarrival(), EventType.IN_EVENT))

    def run_simulation(self):
        self.init_event()
        self.handle_event()
    
    def generate_interarrival(self):
        return np.random.exponential(la)
    
    def generate_service(self):
        return np.random.exponential(mu)

    def sort_events(self):
        return sorted(
                self.events,
                key=lambda time: getattr(time, 'time'))

sim = Simulation()

for _ in range (10): 
    sim.run_simulation()
for event in sim.events:
   #  sim.sort_events()
    print(event.get_event_stats())



{'oc_time': 31.129045443005563, 'client_id': 'c56b7d8f-846f-49ad-992e-4240d65d888d', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 30.62153010229168, 'client_id': 'b909ebc0-387f-4d25-bebc-c0c6b3f7dd3c', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 29.745277140103408, 'client_id': '0d587f31-9b2d-49d6-896f-e72c4c380637', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 28.61851647500425, 'client_id': '2f9376e7-bbb1-40b4-b207-61db937a70c0', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 23.339280585802275, 'client_id': 'cf840968-0088-4b56-beda-4bbcbc2fd176', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 17.923823828429985, 'client_id': 'a001c131-9f47-49e5-9f04-d71eff3d7743', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 15.387385356393473, 'client_id': 'c083ca5b-e791-400c-931b-2bcb0731ec6b', 'event_type': <EventType.OUT_EVENT: 'OUT_EVENT'>}
{'oc_time': 13.67674515079381, 'client_id': '313d5134-9c1

get pobiera zdarzenie z listy z namnijeszym czasem, przesuwa czas symulacji do tego czasu

To do: 
- ogarnać czy potrzebny jest server
    - liczba n ?
- dodac logging do servera lub klientów
- zmienić symualcje od maksymalnego czasu, a nie iteracji 
- zrobić jakieś wykresy od lambda ? 
- zrobić historgram dla b
- sprawko 
- wysłać

UAKTUALNIENIE: ponieważ w punkcie III projektu wkradł się błąd a jest już blisko terminu oddania prac, proszę wykonać tę część przy prostym założeniu, że odstęp miedzy przychodzącymi klientami jest stały i równy średniej z punktu I. 
Reference materials